In [ ]:
%load_ext autoreload
%autoreload 2

import deltas
from deltas.pipeline import data, classifier, evaluation
from deltas.model import downsample
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
# np.random.seed(10)


In [ ]:

costs = (1, 1)  # change for (1, 10) to increase results

datasets = {0: 'Breast Cancer', 2: 'Iris', 3: 'Wine', 4: 'Pima Indian Diabetes',
            5: 'Sonar Rocks vs Mines', 6: 'Banknote Authentication',
            7: 'Abalone Gender', 8: 'Ionosphere', 9: 'Wheat Seeds',
            10: 'Credit Scoring 1', 11: 'Credit Scoring 2',
            12: 'Direct Marketing', 13: 'Habermans breast cancer',
            14: 'Wisconsin Breast Cancer', 15: 'Hepatitis',
            16: 'Heart Disease'}

dataset = datasets[0]  # change ind to select dataset to use
# dataset = datasets[4]  # change ind to select dataset to use
dataset = datasets[14]  # change ind to select dataset to use
# dataset = datasets[15]  # change ind to select dataset to use
dataset = datasets[16]  # change ind to select dataset to use
model = 'SVM-rbf'
# model = 'MLP'
# model = 'Linear'

# dataset = datasets[7]  # change ind to select dataset to use
dfs = []
import random
for i in tqdm(range(10)):
    # np.random.seed(random.randint)
    # np.random.seed(i)
    # data_clf = data.get_real_dataset(dataset, _print=False, seed=i, scale=True)
    data_clf = data.get_real_dataset(dataset, _print=False, scale=True)
    classifiers_dict = classifier.get_classifier(
        data_clf=data_clf,
        model=model,
        _plot=False,
        _print=False)
    data_clf['clf'] = classifiers_dict['Baseline']
    X = data_clf['data']['X']
    y = data_clf['data']['y']
    clf = data_clf['clf']
    # deltas_model = downsample.downsample_deltas(
    #     clf).fit(X, y, _print=True, _plot=True, max_trials=10000)
    # deltas_model = base.base_deltas(
    #     clf).fit(X, y, grid_search=True, _print=True, _plot=True)
    param_grid = {
                #   'alpha': [0, 0.1, 1, 10],
                  'grid_search': [True, False],
                  'method': ['supports-prop-update_mean', 'supports-prop-update_mean-margin_only']}
    grid_original = GridSearchCV(
        downsample.downsample_deltas(), param_grid, refit=True)
    grid_original.fit(X, y,
                      clf=clf,
                      _print=False,
                      _plot=False,
                      max_trials=10000,
                      parallel=True)
    deltas_model = grid_original.best_estimator_
    print(f'Best params: {grid_original.best_params_}')

    if deltas_model.is_fit == True:
        classifiers_dict['Our Method'] = deltas_model
        scores_df = evaluation.eval_test(classifiers_dict,
                                         data_clf['data_test'], _print=False, _plot=False)
        dfs.append(scores_df)
    else:
        print('not fit deltas')

In [ ]:
print(len(dfs))

In [ ]:
df = pd.concat(dfs, axis=0)
mean = {}
std = {}
index = df.index.unique().to_list()
cols = df.columns.to_list()
for method in index:
    mean[method] = df.loc[method].mean().to_list()
    std[method] = df.loc[method].std().to_list()

mean_df = pd.DataFrame.from_dict(mean, orient='index', columns=cols)
std_df = pd.DataFrame.from_dict(std, orient='index', columns=cols)
print(dataset)
print('mean')
print(mean_df)
print('\nstd')
print(std_df)